In [1]:
import numpy as np
import pandas as pd

In [2]:
card = pd.read_csv("card.csv")
adstrd = pd.read_csv("adstrd_master.csv")

In [94]:
# row 최대 출력 갯수 지정

pd.options.display.max_rows=999

In [20]:
# card 데이터 컬럼명 변경

card.columns=['카드사용내역접수일자','행정동코드','행정동명','업종코드','업종명','매출발생건수','매출발생금액']

In [27]:
# 행정동 정보 컬럼명 변경

adstrd.columns=['행정동코드','동','시','구']

In [28]:
# 행정동 코드를 int 타입으로 변경

adstrd.행정동코드=adstrd.행정동코드.astype(int)

In [29]:
# 행정동,시군구 데이터에서 서울 데이터만 추출

adstrd_seoul=adstrd.loc[adstrd.시=="서울특별시"]

In [78]:
# 서울 데이터 컬럼명 변경

adstrd_seoul.columns=['행정동코드','행정동명','시','구']

In [57]:
# card 데이터에서 행정동 코드를 추출하여 string 타입으로 변경
code=card.행정동코드
code=code.astype(str)

# 문자 타입으로 변경한 행정동 코드를 시군구 데이터의 행정동코드와 같은 자릿수로 맞춰줌
for i in range(len(code)):
    code[i]=code[i][:8]

In [71]:
# 바꿔준 행정동 코드를 다시 문자에서 정수 타입으로 변환한 뒤 데이터 프레임에 삽입

code=code.astype(int)
card.행정동코드=code

In [153]:
# 행정동 데이터프레임과 카드 데이터 프레임을 행정동 코드를 기준으로 병합

dong=pd.merge(adstrd_seoul, card, on='행정동코드', how='outer')

In [154]:
# 불필요한 컬럼 제거 및 컬럼명 변경

dong.drop(['행정동명_y'],axis=1, inplace=True)
dong.rename(columns={'행정동명_x':'행정동명'}, inplace=True)

In [125]:
# 일별로 나와있는 카드사용내역 접수일자를 년도와 월로 grouping하기 위해 변경해줌

date=dong['카드사용내역접수일자']
date=date.astype(str)
for i in range(len(date)):
    date[i]=date[i][:6]

In [146]:
# 업종명 출력

dong['업종명'].unique()

array(['세탁소', '기타음료식품', '제과점', '스넥', '주점', '서양음식', '중국음식', '일반한식', '약국',
       '정장', '슈퍼 마켓', '편 의 점', '화   원', '정 육 점', '농축수산품', '미 용 원', '유아원',
       '기타잡화', '페 인 트', '보일러펌프샷시', '기념품 점', '문구용 품', '의원', '정기간행물',
       '피부미용실', '기타수리서비스', '주방용식기', '악 기 점', '치과의원', '안경', '컴 퓨 터',
       '종합용역', '자동차정비', '민예공예품', '기타의료기관및기타의료기기', '스포츠레져용품', '교육테이프',
       '악세 사리', '기계 공구', '주 차 장', '기타레져업', '일식회집', '보습학원', '세 차 장',
       '기타건축자재', '주류판매점', '당구장', '기타의류', '위탁급식업', '예체능학원', '기타대인서비스',
       '노래방', '사우나', '한의원', '맞춤복 점', '학원(회원제형태)', '사무 기기', '기타회원제형태업소4',
       '미용 재료', '사진관', '신   발', '기타숙박업', '칵테일바', '안마/스포츠마사지',
       '서적출판(회원제형태)', '이륜차판매', '미곡상', '화   랑', '일반 서적', '기타가구',
       '비료/농약/사료/종자', '상품권 전문판매', '화물 운송', '문화취미기타', '전문 서적', '테니스 장',
       '기타전문점', '기타용역서비스', '인터넷Mall', '관광여행', '사무서비스', '출판인쇄물', '내의판매점',
       '귀 금 속', '인테 리어', '기타 교육', '통신 기기', '외국어학원', '카 메 라', '양 품 점',
       '유   리', '가정용품수리', '침구수예점', '일반 가구', '인삼 제품', '주방 용구', '완 구 점',
       '화 장 품', '골프연습장', 

In [213]:
# 업종 중, 여행과 관련한 데이터만 출력하여 데이터 프레임으로 생성

travel=dong[dong['업종명'].isin(['관광여행','2급 호텔','렌 트 카','1급 호텔','특급 호텔','고속 버스','콘   도', '철   도','여 객 선'])]

In [214]:
# 추출한 데이터를 구, 업종명, 카드사용내역 접수일자를 기준으로 pivot table 생성

travel=pd.pivot_table(travel, index=['구','업종명','카드사용내역접수일자'], values=['매출발생건수','매출발생금액'], aggfunc='sum')

In [215]:
travel

매출발생건수       매출발생금액
구    업종명   카드사용내역접수일자                     
강남구  1급 호텔 202001        1310    122283476
           202002        2639    234917798
           202003        3127    202968013
           202004        3233    226208170
           202005        3373    287115097
           202006        1461     96838179
     2급 호텔 202001         218     18583030
           202002         480     32451872
           202003         471     33895735
           202004         459     31970584
           202005         591     43605125
           202006         289     17163121
     고속 버스 202001           3       132000
           202002           2       132000
           202003           1        88000
     관광여행  202001        1009    551630901
           202002        1097    465914073
           202003         980     62890264
           202004         893    126397837
           202005         931     98987994
           202006         614    135720135
     렌 트 카 202001         762     71905669
           202002        2273    188645657
           202003        2785    244819284
           202004        2840    261632973
           202005        2961    244738983
           202006        1439    102698476
     철   도 202001      112646   3052133200
           202002      215376   6036412700
           202003      175934   5367566900
           202004      264731   8738923800
           202005      352895  11205047700
           202006      165493   5280171800
     특급 호텔 202001       11871   3313590603
           202002       11978   3434909805
           202003       12758   2150793817
           202004       11506   2610537908
           202005       13930   3667989421
           202006        7113   1476616192
강동구  관광여행  202001           9     11524867
           202002           4      2014900
           202005           4      1850000
           202006           1      2900000
     렌 트 카 202001          65     19498927
           202002         134     82309876
           202003         180     81486800
           202004         186     96413477
           202005         235     95984844
           202006         128     39949605
강북구  렌 트 카 202001          36      4058800
           202002          52     12719850
           202003          69     12498259
           202004          84      8429456
           202005         116     14087550
           202006          51      5050194
강서구  1급 호텔 202001          40      6035636
           202002          68      9170059
           202003          71      3993474
           202004          78      3213400
           202005          80      3421400
           202006          43      1594000
     2급 호텔 202001          45      1557000
           202002         114      4958000
           202003         140      5580000
           202004         123      5080000
           202005         161      7070000
           202006          58      2875000
     고속 버스 202001         941     70815500
           202002        1329     38516000
           202003         421     17079000
           202004           4       120000
           202005           1        90000
     관광여행  202001          14     24243700
           202002          20     49435000
           202003          30     67795868
           202004          39     44965600
           202005          26     26410610
           202006          11      7917400
     렌 트 카 202001          72     18264200
           202002         198     49106768
           202003         222     76355711
           202004         213     49734624
           202005         268     46806534
           202006         150     22066330
     여 객 선 202001       19523    770924482
           202002       31771   1381122779
           202003       48129   1913631603
           202004       57806   2301258887
           202005       79133   3458604081
           202006       35226   1716514552
     특급 호텔 202001         914    169528614
           202002       